# Introduction to Data Science with Julia

<img src="http://julialang.org/images/logo_hires.png" alt="Julia Logo" width="300"></img>

# 目次
- [データ分析入門](#データ分析入門)
 - [データの読み込み](#データの読み込み)
 - [DataFrames パッケージ](#DataFrames-パッケージ)
 - [データ加工](#データ加工)
 - [データの結合](#データの結合)
- [練習問題](#練習問題)

# データ分析入門

## データの読み込み

世にあふれるデータは Excel や CSV, SQL などいろいろな形式で保存されていますが、この講義では取り扱いが容易な CSV (Comma-Separated Values) ファイルを使用していきます。CSV ファイルは中身を見ればわかりますが、各要素がコンマで区切られたテキストファイルです。テキストファイルであるためメモ帳などで簡単に編集することが出来ます。


まずは CSV ファイルからデータを読み込み、平均や分散などの基本統計量を計算してみましょう。
今回はサンプルとして、学生のIDと2つの試験の点数が保存された scores.csv というファイルを用意しました。

Julia の標準機能を使って csv ファイルを読み込むには readcsv 関数を使います。

In [ ]:
scores = readcsv("../data/scores.csv", Int, header=true) # デフォルトでは header=false になっています。

データを読み込むと変数 scores の第1要素が数値データに、第2要素が header が入ります。

In [ ]:
scores[1]

In [ ]:
scores[2]

summarystats を使って2つの試験の点数の平均などを調べてみましょう。

In [ ]:
import StatsBase
StatsBase.summarystats(scores[1][:,2])

In [ ]:
StatsBase.summarystats(scores[1][:,3])

exam 1 と exam 2 の相関係数も計算してみましょう

In [ ]:
corcoeff = cor(scores[1][:,2], scores[1][:,3])

数字だけを見ていてもよくわからないので可視化もしてみましょう。

In [ ]:
import Plots
Plots.gr(leg=false)

In [ ]:
@show a, b = linreg(scores[1][:,2], scores[1][:,3]) # y = a  + b * x にフィッティング
Plots.plot(scores[1][:,2], scores[1][:,3], 
    linetype=:scatter, 
    leg=false,
    title="Scores",
    xlabel="exam 1",
    ylabel="exam 2",
    xticks=0:20:100,
    yticks=0:20:100,
    xlims=(0,100),
    ylims=(0,100),
    aspect_ratio=1
    )
Plots.plot!([0,100], a + b * [0,100])

[目次に戻る](#目次)

## DataFrames パッケージ

先程は readcsv を使ってデータを読み込みましたが、header があるにも関わらず全く活用しませんでした。readcsv で取り込んだ場合、exam1 は 2 列目で、exam2 は 3 列目だから... と header と対応する列番号が必要でした。これではデータ数が増えるとほしいデータ列が何列目なのか数えるだけでも大変です。header があるのだからこれを活用したいものです。

DataFrames パッケージを使うと、各列を header で指定することが出来ます。DataFrames パッケージをつかって CSV ファイルを読み込む場合は readtable を使用します。

(注) [DataFrames.jl](https://github.com/JuliaStats/DataFrames.jl) からの派生で [DataTables.jl](https://github.com/JuliaData/DataTables.jl) というものもあり、今後 DataTables.jl が主流になっていくと思います。

In [ ]:
import DataFrames

In [ ]:
df = DataFrames.readtable("../data/scores.csv")
DataFrames.head(df, 5) # head(df, n)  最初の n 行目までを表示。tail を使うと末尾を表示

読み込んだデータは一見すると配列に似ていますが、実際は DataFrames パッケージ内で定義された DataFrame 型という型で配列と似て非なるものです。概ね配列と同様に扱えますが、扱い方が変わる部分もあるのでそのことを念頭においてください。

In [ ]:
typeof(df)

各列を抜き出すには普通の配列のように
```julia
    df[:,2]
```
とするか、抜き出す列の列番号または header で指定して

```julia
    df[2]
    
    or
    
    df[:exam1]
```
などとします。

In [ ]:
df[:,2]

In [ ]:
df[2]

In [ ]:
df[:exam1]

複数の列を抜き出す場合は
```julia
    df[ [:exam1, :exam2] ] # 内側のカッコは必須
```
のようにします。

In [ ]:
df[ [:exam1, :exam2] ]

describe を使うと各列の平均、五数要約、欠損値の数と割合が標準出力されます。

In [ ]:
DataFrames.describe(df)

既存の配列から DataFrame を作るには次のようにします。

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
tmpdf = DataFrames.DataFrame(X = x, Y = y) #  a = b としたとき、左の a が列名になる。
                                           # 列名と配列の変数名は同じでも構わない。すなわち、a = a と書いてもOK

In [ ]:
tmpdf[:X]

可視化も配列の時同様にすることができます。

In [ ]:
Plots.plot(df[:exam1], df[:exam2],
    linetype=:scatter, 
    leg=false,
    title="Scores",
    xlabel="exam 1",
    ylabel="exam 2",
    xticks=0:20:100,
    yticks=0:20:100,
    xlims=(0,100),
    ylims=(0,100),
    aspect_ratio=1)

Plots にさらに箱ひげ図などの描写機能を加え、DataFrame 型にも対応している StatPlots パッケージを使うとより分析がはかどります。

In [ ]:
import StatPlots # 文法は Plots と概ね一緒
StatPlots.gr()

普通の Plots では出来きませんが、StatPlots は DataFrame 型に対応しているので次のようにも書けます

In [ ]:
StatPlots.plot(df, :exam1, :exam2, linetype=:scatter) # plot(df[:exam1], df[:exam2]) の代わりに左のように書ける

In [ ]:
StatPlots.histogram(df, :exam1, bins=0:5:100, label="exam 1")
StatPlots.histogram!(df, :exam2, bins=0:5:100, alpha=0.5, label="exam 2", leg=true) # alpha は透過度
StatPlots.xlabel!("score")

In [ ]:
StatPlots.boxplot(df, ["exam 1" "exam 2"], :exam1, :exam2)
StatPlots.ylabel!("score")

[目次に戻る](#目次)

## データ加工

現在では[統計局](http://www.stat.go.jp/data/guide/download/)や [Kaggle](https://www.kaggle.com/datasets) などから様々なデータをダウンロード出来ますが、それらのデータが始めから解析しやすい形式になっているとは限りません。そのため、まずは解析がしやすくするためにデータを加工する必要が有ります。

まずは ID、性別、誕生日だけが入った単純なデータ people.csv を使ってデータ加工に慣れていきましょう。

In [ ]:
people = DataFrames.readtable("../data/people.csv")
DataFrames.head(people)

各列の要素の型を調べてみます。

In [ ]:
DataFrames.eltypes(people)

ID 列は Int64 で、性別と誕生日の列は String 型のようです。Julia には日付を扱うための Date 型があるので、より Julia で扱いやすいように誕生日は String 型から Date 型 へ変換したくなります。性別の方も書き方が何通りかあるようなので表し方を揃えたいところです。

初めに変換が簡単そうな誕生日の方からやっていきましょう。今回のように「年-月-日」という日付の表し方だと Date を使うだけで簡単に Date 型になります。

In [ ]:
Date(people[:Birthday][1])

ヨーロッパ式に 日・月・年 の順に書かれていた場合は次のように書きます。

In [ ]:
Date("12-31-2020", "m-d-y")

一見すると変換されていないようですが、型を調べるとちゃんと型変換がされています。

In [ ]:
@show typeof(people[:Birthday][1])
@show typeof(Date(people[:Birthday][1]));

Date 型にすれば、2つの日付の引き算をすると経過日数を計算することが出来ます。

In [ ]:
yourbirthday = "2000-1-1"
print("今日はあなたが生まれてから ", Dates.today() - Date(yourbirthday), " です。")

これから、誕生日の列を String 型から Date 型に変換していきますが、もともとの誕生日の列は String 型なので、変換した Date 型を直接代入することは出来ません。そのため、ここでは
1. birthday という列を新しく作る
1. birthday 列に変換後の誕生日を入れる
1. 元の Birthday 列を削除
1. birthday を Birthday に名前を変更

という手順を踏んでいくことにします。

In [ ]:
# 新しい列を作成
# 初期化していないので各要素の日付に意味はありません。
people[:birthday] = Array{Date}(size(people, 1))

In [ ]:
# 型を変換し代入する
for i in 1:size(people, 1)
    people[:birthday][i] = Date(people[:Birthday][i])
end
DataFrames.head(people)

In [ ]:
# Birthday 列を削除
delete!(people, :Birthday)
DataFrames.head(people)

In [ ]:
# 列名を変更
DataFrames.rename!(people, :birthday, :Birthday)
DataFrames.head(people)

In [ ]:
# 無事に変換できたか確認
DataFrames.eltypes(people)

注) 今回の様に、変換する列に欠損値が存在しない場合、新しく列を作らずとも一気に型変換できます。

In [ ]:
# people = readtable("people.csv")
# people[:Birthday] = Date(people[:Birthday])
# @show head(people)
# eltypes(people)

次に性別の方を加工していきましょう。性別の列を見るとどうやら男女の書き方が何通りかあるようです。

In [ ]:
people[:Sex]

このままでは、何通りの書き方があるのかはっきりしないので、Set を使ってこの列から重複する要素を取り除きましょう。<br>
Set 型は数学の集合の同様、要素の順序に意味はありません。

In [ ]:
Set(people[:Sex])

これから、男性は M, Male, male の3通りの書き方が、女性は F, Female, female の3通りの書き方があることがわかります。 <br>
3通りもあると扱いが大変なので、ここでは男性は M、女性は F で統一していきましょう。

この性別の列の加工は皆さんが実際にコードを書いてみてください。

ヒント
```julia
    x in 配列 (または集合)
        or
    x ∈ 配列 (または集合)
```
とすると x が配列の中に含まれるかどうかを確認することが出来る。

In [ ]:
@show 1 in [1, 2, 3]
@show 5 ∈ [1, 2, 3];

In [ ]:
# 実際にコードを書いてみてください


加工ができたら最後に加工したデータを保存しましょう。

In [ ]:
# 元のデータと同じ名前にすると上書きされてしまうので、必ず違うファイル名をつける。
DataFrames.writetable("people_fix.csv", people)

上記の方法で保存したファイルをもう一度読み込むと誕生日の列はまた String 型で読み込まれます。そのため扱いやすくするにはまた Date 型へまた変換する必要が有ります。

せっかく扱いやすい型に加工したのだから、その型のまま保存・読み込みがしたいものです。そのようなときには[JLD.jl](https://github.com/JuliaIO/JLD.jl) パッケージを使うと型の情報を保持したまま比較的楽に変数を保存することが出来ます。

In [ ]:
import JLD

In [ ]:
# 変数を保存
JLD.@save "people_fix.jld" people

# 読み込むときは
# JLD.@load "people_fix.jld"

[目次に戻る](#目次)

## データの結合

まずは変数の保存が正しく行われたか確認するために一度カーネルを restart して、下記のコマンドを実行して変数を読み込んでみましょう。

カーネルの restart の方法は上の Kernel のタブをクリックし、Restart をクリックしてください。これを行うと Julia を再起動したのと同じになります。そのため、今まで計算してきた変数などの情報は失われます。

In [ ]:
import JLD, DataFrames
JLD.@load "people_fix.jld"
DataFrames.head(people)

In [ ]:
DataFrames.eltypes(people)

さらに、今回は people_satisfaction.csv という CSV ファイルも読み込みます。この people_satisfaction.csv 中には
ID、あるサービスに対する満足度 の2つのデータが入っています。ここで、people.csv と people_satisfaction.csv の ID が同じ人は同一人物だとします。

満足度は数値で入っており
1. 非常に不満
1. 不満
1. 普通
1. 満足
1. 非常に満足

を表します。



In [ ]:
people_satisfaction = DataFrames.readtable("../data/people_satisfaction.csv")
DataFrames.head(people_satisfaction)

次に people と people_satisfaction のデータを結合していきます。今回は共通項目として ID があるのでこれを基準にして結合します。
2つのデータを結合する場合には join 関数を使います。

In [ ]:
people = DataFrames.join(people, people_satisfaction, on = :ID, kind = :outer)
DataFrames.head(people)

kind を変えると結合方法が変わります。詳しくは[公式ドキュメント](http://juliastats.github.io/DataFrames.jl/stable/man/joins/)を読んでください。

今回の様に ID が 2 つのファイルで完全に同じ場合、
```julia
people[:Satisfaction] = sort(people_satisfaction)[:Satisfaction]
```
として代入することも出来ますが、常に一致する ID があるとは限らないので join 関数を使ったほうが無難です。

[目次に戻る](#目次)

## 欠損値

結合結果を見ると ID 2 の人の Satisfaction が NA となっていることがわかります。ここで NA とは欠損値を表します。アンケートで無回答だった場合や、実験でサンプルが取れなかった場合は欠損値になります。

欠損値がある場合、単純に平均などを求めることが出来ません。

In [ ]:
mean(people[:Satisfaction])

データから欠損値を取り除くには dropna を使います。

In [ ]:
DataFrames.dropna(people[:Satisfaction])

In [ ]:
mean(DataFrames.dropna(people[:Satisfaction]))

欠損値があるデータを describe を使ってみると、欠損値を取り除いたデータでの平均などが表示されます。

In [ ]:
DataFrames.describe(people)

describe の出力結果より Satisfaction には欠損値が 102 個あることがわかります。
この欠損値をどう扱うかというのは難しい問題ですが、ここでは欠損値を平均値に置き換えてみます。

In [ ]:
people[:Satisfaction_fix] = Vector{Float64}(size(people, 1))
for iter in 1:size(people, 1)
    if DataFrames.isna(people[:Satisfaction][iter])
        people[:Satisfaction_fix][iter] = mean(DataFrames.dropna(people[:Satisfaction]))
    else
        people[:Satisfaction_fix][iter] = people[:Satisfaction][iter]
    end
end
DataFrames.head(people)

要素が NA か否かを調べるときには isna 関数を使います。isna 関数は引数が NA ならば true を、そうでなければ false を返します。

上の例では for 文を使って欠損値を平均値に置き換えましたが、convert を使うと欠損値をコード一行書きで置き換えることが出来ます。

In [ ]:
# meansatisfaction = mean(DataFrames.dropna(people[:Satisfaction]))
# people[:Satisfaction_fix] = convert(Vector{Float64}, people[:Satisfaction], meansatisfaction)

[目次に戻る](#目次)

# 練習問題

## 1.

people から年齢のヒストグラムを作成せよ。ただし、年齢は数え年とする。

## 2.

2つのデータセット names, jobs を ID を基準に結合せよ。<br>
また、kind オプションを変えるとどのような結果が得られるのか確認せよ。<br>
※ オプションは[公式ドキュメント](http://juliastats.github.io/DataFrames.jl/stable/man/joins/#Database-Style-Joins-1)参照

```julia
using DataFrames
names = DataFrame(ID = [1, 2], Name = ["John Doe", "Jane Doe"])
jobs = DataFrame(ID = [1, 3], Job = ["Lawyer", "Doctor"])
```



## 3.

下記のデータの欠損値を全て中央値に置き換えよ
```julia
srand(1)
ex = DataFrame()
ex[:sample] = @data([rand(@data([NA, 1, 2, 3, 4, 5])) for i in 1:100])
```

[目次に戻る](#目次)